# Simple example of a homogeneous ensemble using learning networks

In this simple example, no bagging is used, so every atomic model
gets the same learned parameters, unless the atomic model training
algorithm has randomness, eg, DecisionTree with random subsampling
of features at nodes.

In [1]:
using MLJ
using Plots; pyplot()

Plots.PyPlotBackend()

learning network (composite model spec):

In [2]:
Xs = source()
ys = source(kind=:target)

atom = @load DecisionTreeRegressor
atom.n_subfeatures = 4 # to ensure diversity among trained atomic models

machines = (machine(atom, Xs, ys) for i in 1:100)

Base.Generator{UnitRange{Int64},Main.##471.var"#1#2"}(Main.##471.var"#1#2"(), 1:100)

overload summation for nodes:

In [3]:
Base.sum(v...) = sum(v)
Base.sum(v::AbstractVector{<:AbstractNode}) = node(sum, v...)

yhat = sum([predict(m, Xs) for  m in machines]);

new composite model type and instance:

In [4]:
one_hundred_models = @from_network OneHundredModels(atom=atom) <= yhat

Main.##471.OneHundredModels(atom = DecisionTreeRegressor(pruning_purity_threshold = 0.0,
                                                         max_depth = -1,
                                                         min_samples_leaf = 5,
                                                         min_samples_split = 2,
                                                         min_purity_increase = 0.0,
                                                         n_subfeatures = 4,
                                                         post_prune = false,),) @ 7…01

application to data

In [5]:
X, y = @load_boston;

tune regularization parameter for a *single* tree:

In [6]:
r = range(atom,
          :min_samples_split,
          lower=2,
          upper=100, scale=:log)

mach = machine(atom, X, y)

curve = learning_curve!(mach,
                        range=r,
                        measure=mav,
                        resampling=CV(nfolds=9))

plot(curve.parameter_values, curve.measurements, xlab=curve.parameter_name)
savefig("atom.png")

┌ Info: Training Machine{DeterministicTunedModel} @ 7…10.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/machines.jl:172
Iterating over a 26-point grid: 100%[=========================] Time: 0:00:00
┌ Info: Training of best model suppressed.
│  To train tuning machine `mach` on all supplied data, call `fit!(mach.fitresult)`.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/tuning.jl:316


tune regularization parameter for all trees in ensemble simultaneously:

In [7]:
r = range(one_hundred_models,
          :(atom.min_samples_split),
          lower=2,
          upper=100, scale=:log)

mach = machine(one_hundred_models, X, y)

curve = learning_curve!(mach,
                        range=r,
                        measure=mav,
                        resampling=CV(nfolds=9))

plot(curve.parameter_values, curve.measurements, xlab=curve.parameter_name)
savefig("ensemble.png")

┌ Info: Training Machine{DeterministicTunedModel} @ 5…93.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/machines.jl:172
Iterating over a 26-point grid: 100%[=========================] Time: 0:00:17
┌ Info: Training of best model suppressed.
│  To train tuning machine `mach` on all supplied data, call `fit!(mach.fitresult)`.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/tuning.jl:316


*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*